# Task 1
The US

### Task 2.2: WA Counties with the Largest Populations

In [111]:
import pandas as pd
import geopandas

#read files into geopandas
waShape = geopandas.read_file("saep_bg10/saep_bg10.shp")
countyName = geopandas.read_file("WashingtonFIPS.dbf")

#join the shapefile and county name file together
wa_st = pd.merge(waShape, countyName, left_on = ['COUNTYFP10'], right_on = ['FIPSCounty'], how = 'left')

#create dataframe of countyname and 2017 population
cnty_pop = wa_st[['CountyName', 'POP2017']]

#additional functions to dataframe
cnty_pop = cnty_pop.groupby(['CountyName'])
cnty_pop = cnty_pop.sum()
cnty_pop = cnty_pop.sort_values(by = ['POP2017'], ascending=False)
cnty_pop.columns = ['Population (2017)']
cnty_pop.index.names = ['County Name']

#print out county dataframe of county name and population
cnty_pop.head(10)


,Population (2017)
County Name,
King,2153700.0
Pierce,859400.0
Snohomish,789400.0
Spokane,499800.0
Clark,471000.0
Thurston,276900.0
Kitsap,264300.0
Yakima,253000.0
Whatcom,216300.0


In [19]:
#PART 2. CREATES SHAPEFILE FOR EACH COUNTY
import sys
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.6\\bin')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.6\\arcpy')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.6\\ArcToolbox\\Scripts')
import arcpy

#create workspace
arcpy.env.workspace="C:\\Users\\Ginue Han\\geog458_assignment\\lab3"

#join saep_bg10.shp and WashingtonFIPS by county FIPS. add the county name field to shape file.
arcpy.JoinField_management("saep_bg10\\saep_bg10.shp","COUNTYFP10", "WashingtonFIPS.dbf", "FIPSCounty",["CountyName"])

#create a list with the fields names of the shapefile
field_names = [str(f.name) for f in arcpy.ListFields("saep_bg10\\saep_bg10.shp")]

#list to store all county names in WA
county_names =[]
#create cursor for shapefile to loop through each row
cursor = arcpy.da.SearchCursor("saep_bg10\\saep_bg10.shp", "CountyName")
#uses cursor to loop through each row in shapefile inorder to add unique countynames into countynames list
for row in cursor:
    if str(row[0]) not in county_names:
        county_names.append(str(row[0]))
del cursor

#loops through county names and uses the Select_analysis tool to select rows from saep_bg10 shapefile that are from the currently
#selected county and creates a new shapefile for that county. 
for county in county_names:
    in_features = "saep_bg10\\saep_bg10.shp"
    out_feature_class = "CountyData\\{0}.shp".format(county)
    where_clause='"CountyName" = \'{0}\''.format(county)
    arcpy.Select_analysis(in_features, out_feature_class, where_clause)

ImportError: No module named arcpy

# Part 3 Urban vs. Rural

In [144]:
# PART 3. Urban vs. Rural

#3.1     
#Add column to give each block value of urban or rural based on 2017
for index, row in wa_st.iterrows():
    if row['POP2017']>1921.638:
        wa_st.loc[index,'Urban/Rural']='Urban'
    else:
        wa_st.loc[index,'Urban/Rural']='Rural'
wa_st

#3.2
#Caclulates and prints out what percentage of population is urbanized for 2017
#Sum up the populations that are 'Urban'
urbanCount = (float)((wa_st['Urban/Rural'] =='Urban').sum())
#Sum up the populations that are 'Rural'
ruralCount = (float)((wa_st['Urban/Rural'] =='Rural').sum())
#Calculate the percentage of the population urbanized
percent = 100.00 *( urbanCount / (urbanCount+ruralCount))
print 'Percentage of population urbanized in 2017 is %.2f percent' %  percent

#3.3
#Calculates and prints out what percentage of land area is urbanized for 2017
#Making dataframe of only rows that are 'Urban', then sum up the land area 
urban = wa_st['Urban/Rural'] == 'Urban'
wa_st_urban = wa_st[urban]
urbanCountLA = (float)((wa_st_urban['ALANDMI']).sum())
#Making dataframe of only rows that are 'Rural', then sum up the land area 
rural = wa_st['Urban/Rural'] == 'Rural'
wa_st_rural = wa_st[rural]
ruralCountLA = (float)((wa_st_rural['ALANDMI']).sum())
#Calculate total land area of urbanized and the percentage of it
percentLA = 100.00 *( urbanCountLA / (urbanCountLA+ruralCountLA))
print 'Percentage of land area urbanized in 2017 is %.2f percent' %  percentLA

#3.4
#Add column to the shapefile datatable for whether block groups have urbanized, no change in category, or deurbanized: 2007 v 20017 
for index, row in waShape.iterrows():
   # value = row['POP2017'] - row['POP2007'] >1921.638
    if  row['POP2007'] >= 1464.438: #urban in 2007 
        if row['POP2017'] >= 1921.638: #urban in 2017 
            waShape.loc[index,'Urban/Rural 2007-2017']='no change in category'
        else: #not urban in 2017 
            waShape.loc[index,'Urban/Rural 2007-2017']='deurbanized'
    else: #not urban in 2007 
        if row['POP2017'] >= 1921.638: #urban in 2017 
            waShape.loc[index,'Urban/Rural 2007-2017']='urbanized'
        else: #not urban in 2017 
            waShape.loc[index,'Urban/Rural 2007-2017']='no change in category'
waShape

#5. Write code that calculates and prints out how many block groups urbanized and how many deurbanized over the previous decade.
urbanCountChange = (float)((waShape['Urban/Rural 2007-2017'] =='urbanized').sum())
deurbanCountChange = (float)((waShape['Urban/Rural 2007-2017'] =='deurbanized').sum())
percentChangeUrban = 100.00 *( urbanCountChange/ (urbanCountChange+deurbanCountChange))
print 'Percentage of block groups that urbanized from 2007 - 2017 is %.2f percent' %  percentChangeUrban
percentChangeDeurban = 100.00 *( deurbanCountChange/ (urbanCountChange+deurbanCountChange))
print 'Percentage of block groups that deurbanized from 2007 - 2017 is %.2f percent' %  percentChangeDeurban

Percentage of population urbanized in 2017 is 21.26 percent
Percentage of land area urbanized in 2017 is 14.71 percent
Percentage of block groups that urbanized from 2007 - 2017 is 8.70 percent
Percentage of block groups that deurbanized from 2007 - 2017 is 91.30 percent
